In [92]:
import os 
from pathlib import Path
import sys
import subprocess
import time
from typing import Optional
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import display
import latextable
from texttable import Texttable
from arguments_dict import arguments_dict

executables = ['genmc-old', 'genmc-wkmo', 'genmc', 'genmc-imm', 'genmc-xmm']
executable_paths_dict = dict([(exe, Path("executables") / exe) for exe in executables])
tests = dict([(t, Path('synthetic') / t / 'variants' / (os.listdir(Path('synthetic') / t / 'variants')[0])) for t in os.listdir('synthetic') if os.path.isdir(Path('synthetic') / t)])
subprocess_timeout = 60 #s (takes 20 mis with 60s timeout)
test_arguments = {
    "reorder2": "",
    "singleton": "",
    "fib_bench": "NUM=4",
    "szymanski#1": "N=1",
    "dekker-bnd": "",
    "indexer#12": "N=12",
    "ainc#4": "N=4", 
    "casrot#7": "N=7",
    "casw#4": "N=4",
    "inc#3": "N=3",
    "rinc#4": "N=4",
    "rw#3": "N=3",
    "wainc#4": "N=4",
}

class RunExecutableResult:
    class Timeout:
        pass
    class Ok:
        def __init__(self, output) -> None:
            super().__init__()
            self.output = output
    class Error:
        def __init__(self, retcode, output) -> None:
            super().__init__()
            self.retcode = retcode
            self.output = output

def run_executable(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    try:
        result = subprocess.run([exe_path, *args, test_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=subprocess_timeout)
        result_stdout = result.stdout.decode()
        result_stderr = result.stderr.decode()
        if result.returncode == 0:
            return RunExecutableResult.Ok(result_stdout + result_stderr)
        else:
            return RunExecutableResult.Error(result.returncode, result_stdout + result_stderr)
    except subprocess.TimeoutExpired:
        return RunExecutableResult.Timeout()

In [93]:
def get_execution_time(string: str) -> Optional[float]:
    try:
        pattern = "Total wall-clock time: "
        time = string.split(pattern)[1].split('s\n')[0]
        return float(time)
    except:
        return None
assert(get_execution_time(
"""
No errors were detected.
Number of complete executions explored: 3
Total wall-clock time: 0.05s
""") == 0.05)

def get_number_executions(string: str) -> Optional[int]:
    pattern = "Number of complete executions explored: "
    i = string.index(pattern) + len(pattern)
    number_str = ""
    while i < len(string) and string[i].isdigit():
        number_str += string[i]
        i += 1
    try:
        return int(number_str)
    except:
        return None

assert(get_number_executions("abc 123 Number of complete executions explored: 123456") == 123456)

def get_duplicate_executions(string: str) -> Optional[int]:
    try:
        pattern = "Number of duplicate executions explored: "
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_duplicate_executions("abc 123 Number of duplicate executions explored: 123456") == 123456)

def get_number_lb_races(string: str) -> Optional[int]:
    pattern = "Number of load buffering races explored: "
    try:
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_number_lb_races("abc 123 Number of load buffering races explored: 123456") == 123456)

class ResultItem:
    def __init__(self, execs, exe_time, dups, lb_races) -> None:
        self.execs = execs
        self.exe_time = exe_time
        self.dups = dups
        self.lb_races = lb_races

def run_and_get_results(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    res = run_executable(exe_path, args, test_path)
    if isinstance(res, RunExecutableResult.Ok):
        execs = get_number_executions(res.output)
        exe_time = get_execution_time(res.output)
        dups = get_duplicate_executions(res.output)
        lb_races = get_number_lb_races(res.output)
        if execs is None:
            # print(f"could not get number of executions in {res.output}")
            execs = 0
        if dups is None:
            dups = 0
        if exe_time is None:
            # print(f"could not get duplicate executions in {res.output}")
            exe_time = 0
        return RunExecutableResult.Ok(ResultItem(execs, exe_time, dups, lb_races))
    else:
        return res

timeout = '{\\fontspec{Symbola}\\symbol{"231B}}'

In [94]:
def run_and_get_results_with_n(executables: list[str], test_name: str, n: int, rows: list[list]) -> list[RunExecutableResult]:
    rows.append([])
    rows[-1] += [f"{test_name}({n})"]
    for exe in executables:
        args = arguments_dict[exe] + ["--"] + [f"-DN={n}"]
        res = run_and_get_results(executable_paths_dict[exe], args, tests[test_name])
        if isinstance(res, RunExecutableResult.Ok):
            execs = res.output.execs
            exe_time = res.output.exe_time
            rows[-1]  += [execs, f"{exe_time:.2f}s"]
            if res.output.lb_races is not None:
                rows[-1] += [res.output.lb_races]
        elif isinstance(res, RunExecutableResult.Timeout):
            rows[-1]  += [timeout] * 2
            rows.pop()
            return rows
        elif isinstance(res, RunExecutableResult.Error):
            print(f"{exe} {test_name} {args} error {res.retcode}: {res.output[:1000]}")
            rows.pop()
            return rows
        else:
            assert(0)
    return run_and_get_results_with_n(executables, test_name, n+1, rows)

In [95]:
rows = [["Test Name", "Old GenMC Execs",  "Old GenMC Time", "WMC Execs", "WMC Time", "GenMC Execs", "GenMC Time", "IMM Execs", "IMM Time", "XMM Execs", "XMM Time", "LB races"]]
assert(executables == ['genmc-old', 'genmc-wkmo', 'genmc', 'genmc-imm', 'genmc-xmm'])
for test_name_with_param in test_arguments.keys():
    if len(test_name_with_param.split("#")) == 2:
        test_name, n = test_name_with_param.split("#")
        run_and_get_results_with_n(executables, test_name, int(n), rows)
    else:
        test_name, = test_name_with_param.split("#")
        row = [f"{test_name_with_param.split('#')[0]}({test_name_with_param.split('#')[1]})" if len(test_name_with_param.split('#')) > 1 else test_name_with_param]
        for exe in executables:
            args = arguments_dict[exe] + ["--"] + [f"-D{a}" for a in test_arguments[test_name_with_param].split()]
            res = run_and_get_results(executable_paths_dict[exe], args, tests[test_name])
            if isinstance(res, RunExecutableResult.Ok):
                execs = res.output.execs
                exe_time = res.output.exe_time      
                row += [execs, f"{exe_time:.2f}s"]
                if res.output.lb_races is not None:
                    row += [res.output.lb_races]
            elif isinstance(res, RunExecutableResult.Timeout):
                print(f"{exe} {test_name_with_param} {test_name} {args} timed out")
                row += [timeout] * 2
            elif isinstance(res, RunExecutableResult.Error):
                print(f"{exe} {test_name_with_param} {test_name} {args} error {res.retcode}: {res.output[:1000]}")
                # row += [res.output[:60] if res.output != "" else f"error code {res.retcode}"] * 2
                row = None
                break
            else:
                assert(0)
        rows.append(row) if row is not None else None

# assert(len(rows) == 16)
print("rows has len", len(rows))

genmc-xmm ainc ['--v1', '--disable-sr', '--XMM', '--', '-DN=7'] error 7: ERROR: BUG: Failure at src/RC11Driver.cpp:1974/getCoherentPlacings()!

genmc-xmm rinc ['--v1', '--disable-sr', '--XMM', '--', '-DN=7'] error 7: ERROR: BUG: Failure at src/RC11Driver.cpp:1974/getCoherentPlacings()!

rows has len 30


In [96]:
data = rows[1:]

for i in range(len(data)):
    data[i][0] = data[i][0].replace("_", "-")

# data.sort(key=lambda d: int(d[-1]) if d[-1] != timeout else -1, reverse=True)

rows = [rows[0]] + data

In [97]:
titles = ["Test Name", "Old GenMC$_\\RCMM$", "WMC", "GenMC$_\\RCMM$", "GenMC$_\\IMM$", "GenMC-XMM", "LB Races"] 
header = [[" ", "\\multicolumn{6}{c|}{Number of Executions}", "", "", "", "", ""], titles]

exec_rows = header + [[row[0]] + row[1:-1][0::2] + [row[-1]] for row in rows[1:]] 

for i in range(2, len(exec_rows)):
    execs = exec_rows[i][1]
    if exec_rows[i][5] != exec_rows[i][3] and exec_rows[i][5] != timeout and exec_rows[i][3] != timeout:
        exec_rows[i][0] = f"\\textcolor{{blue}}{{{exec_rows[i][0]}}}"
    elif any(p != timeout and p != execs for p in exec_rows[i][1:-1]):
        exec_rows[i][0] = f"\\textcolor{{red}}{{{exec_rows[i][0]}}}"

table_2 = Texttable()
table_2.set_cols_align(["l"] + ["c" for i in range(len(exec_rows[0]) - 1)])
table_2.set_cols_valign(["m" for i in range(len(exec_rows[0]))])
table_2.add_rows(exec_rows)

In [98]:
header = [[" ", "\\multicolumn{6}{c|}{Execution Time}", "", "", "", "", ""], titles]

time_rows = header + [[row[0]] + row[2:-1][0::2] + [row[-1]] for row in rows[1:]]

for i in range(len(time_rows)):
    x = time_rows[i]
    if x[-1] == timeout:
        time_rows[i].append(timeout)
    assert(len(x) == len(time_rows[0]))

for i in range(1, len(time_rows)):
    time_rows[i][0] = exec_rows[i][0]

table_1 = Texttable()
table_1.set_cols_align(["l"] + ["c" for i in range(len(time_rows[0]) - 1)])
table_1.set_cols_valign(["m" for i in range(len(time_rows[0]))])
table_1.add_rows(time_rows)

In [99]:
def adjust_header(table: str) -> str:
    return table.replace("\\multicolumn{6}{c|}{Number of Executions} &  &  &  &  &", "\\multicolumn{6}{c|}{Number of Executions}").replace("\\multicolumn{6}{c|}{Execution Time} &  &  &  &  &", "\\multicolumn{6}{c|}{Execution Time}")

def adjust_width(table: str) -> str:
    return table.replace("\\begin{center}", "\\begin{center} \\resizebox{\\textwidth}{!}{").replace("\\end{tabular}", "\\end{tabular}}")

table_2_code = latextable.draw_latex(table_2, position='H', caption=f'Number of executions explored on synthetic benchmarks run with the version of GenMC that WMC is based on ("Old GenMC$_\\RCMM$"), WMC, GenMC$_\\RCMM$, GenMC$_\\IMM$, and GenMC-XMM. Timeout was set to {subprocess_timeout}s", label="table:synthetic-benchmarks-execs')
table_2_code = adjust_width(adjust_header(table_2_code))

table_1_code = latextable.draw_latex(table_1, position='H', caption=f'Execution time on synthetic benchmarks run with the version of GenMC that WMC is based on ("Old GenMC$_\\RCMM$"), WMC, GenMC$_\\RCMM$, GenMC$_\\IMM$, and GenMC-XMM. Timeout was set to {subprocess_timeout}s", label="table:synthetic-benchmarks-time')
table_1_code = adjust_width(adjust_header(table_1_code))

print(table_2_code)
print(table_1_code)


\begin{table}[H]
	\begin{center} \resizebox{\textwidth}{!}{
		\begin{tabular}{|l|c|c|c|c|c|c|}
			\hline
			  & \multicolumn{6}{c|}{Number of Executions}  \\
			\hline
			Test Name & Old GenMC$_\RCMM$ & WMC & GenMC$_\RCMM$ & GenMC$_\IMM$ & GenMC-XMM & LB Races \\
			\hline
			reorder2 & 1296 & 1296 & 1296 & 1296 & 1296 & 0 \\
			\hline
			singleton & 24 & 24 & 24 & 24 & 24 & 0 \\
			\hline
			fib-bench & 34205 & 34205 & 34205 & 34205 & 34205 & 0 \\
			\hline
			\textcolor{blue}{szymanski(1)} & 40 & 44 & 32 & 32 & 37 & 42 \\
			\hline
			\textcolor{blue}{szymanski(2)} & 5738 & 6344 & 2216 & 2216 & 6925 & 27727 \\
			\hline
			\textcolor{red}{dekker-bnd} & 59 & 59 & 55 & 55 & 55 & 123 \\
			\hline
			indexer(12) & 8 & 8 & 8 & 8 & 8 & 0 \\
			\hline
			indexer(13) & 64 & 64 & 64 & 64 & 64 & 0 \\
			\hline
			indexer(14) & 512 & 512 & 512 & 512 & 512 & 0 \\
			\hline
			indexer(15) & 4096 & 4096 & 4096 & 4096 & 4096 & 0 \\
			\hline
			\textcolor{blue}{ainc(4)} & 24 & 24 & 24 & 24 & 26 & 1

In [100]:
print(table_1.draw())

+-----------+-----------+-------+-----------+-----------+-----------+----------+
|           | \multicol |       |           |           |           |          |
|           | umn{6}{c| |       |           |           |           |          |
|           | }{Executi |       |           |           |           |          |
|           | on Time}  |       |           |           |           |          |
+===========+===========+=======+===========+===========+===========+==========+
| Test Name | Old GenMC |  WMC  | GenMC$_\R | GenMC$_\I | GenMC-XMM | LB Races |
|           | $_\RCMM$  |       |   CMM$    |    MM$    |           |          |
+-----------+-----------+-------+-----------+-----------+-----------+----------+
| reorder2  |   0.12s   | 0.12s |   0.14s   |   0.20s   |   0.20s   |    0     |
+-----------+-----------+-------+-----------+-----------+-----------+----------+
| singleton |   0.01s   | 0.01s |   0.04s   |   0.05s   |   0.05s   |    0     |
+-----------+-----------+---

In [101]:
print(table_2.draw())

+-----------+-----------+-------+-----------+-----------+-----------+----------+
|           | \multicol |       |           |           |           |          |
|           | umn{6}{c| |       |           |           |           |          |
|           | }{Number  |       |           |           |           |          |
|           | of Execut |       |           |           |           |          |
|           |   ions}   |       |           |           |           |          |
+===========+===========+=======+===========+===========+===========+==========+
| Test Name | Old GenMC |  WMC  | GenMC$_\R | GenMC$_\I | GenMC-XMM | LB Races |
|           | $_\RCMM$  |       |   CMM$    |    MM$    |           |          |
+-----------+-----------+-------+-----------+-----------+-----------+----------+
| reorder2  |   1296    | 1296  |   1296    |   1296    |   1296    |    0     |
+-----------+-----------+-------+-----------+-----------+-----------+----------+
| singleton |    24     |  2